In [1]:
import pandas as pd
import tabula

In [2]:
# empty list and set for appending timetables and place names.

time_tables = []
places = []

In [3]:
# extracting tables from pdf using tabula

for page in range(1166):
    try:
        
        df = tabula.read_pdf('RMS-EKM-to upload.pdf',pages=str(page),lattice=True)

        # somehow the extraction contains a list of two duplicated dataframes of the table. Therefore selecting the first element of the list.
        df = df[0]

        # the dataframe contains a lot of noise, for e.g., null columns with no headers. So dropping them.
        df = df.dropna(axis=1,how='all')

        # the column header has shifted to the first row and sub-header to the second row. 

        # edit: also it was noticed the final data contained '\r' in both columns and data. So removing it from the dataframe.
        df = df.replace('\r',' ',regex=True)

        # Extracting the names of the places from first row (header row). Removing 'Trip','Day', and null value
        lst_places = list(df.loc[0,:].dropna().unique())
        for i in ['Trip','Day']:
            if i in lst_places:
                lst_places.remove(i)

        # appending place name to places master list 
        for i in lst_places:
            if i not in places:
                places.append(i)
        
        # extracting only the body of the data as dataframe since the headers and sub-headers are messed up.
        # we don't need 'Trip' column since we already have the index
        df = df.iloc[2:,1:]
                
        # index is corrected so as to start from 0.
        df.index-=2

        # a dictionary will be created with 'Day'+places as keys and each place having a value:dict containing 'A','D','Via' as keys
        # inturn 'A','D','Via' will have values:list(columnwise data)
        dict1 = {'Day':list(df.iloc[:,0])}
        column_number=1
        for i in lst_places:
            dict2={}
            for j in ['A','D','Via']:
                values = df.iloc[:,column_number]

                # edit: the time data was found to have '.' instead of ':'
                if j=='A' or j=='D':
                    values = values.str.replace('.',':')
                dict2[j]=list(values)
                column_number+=1
                dict1[i]=dict2
                    
        # appending dictionary to time_tables master list 
        time_tables.append(dict1)
        
    except Exception as e:
        print('Exception',e)

Exception java.lang.IndexOutOfBoundsException: Page number does not exist.
Exception single positional indexer is out-of-bounds
Exception single positional indexer is out-of-bounds
Exception 0
Exception 0
Exception 0
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception 0
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception 0
Exception 0
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception 0
Exception single positional indexer is out-of-bounds
Exception 0
Exception single positional indexer

In [4]:
len(time_tables)

1108

In [5]:
len(places)

566

In [6]:
# converting and exporting time_tables list as JSON file

import json

with open("time_tables.json", "w") as outfile: 
            json.dump(time_tables, outfile)

In [7]:
# converting and exporting places list as JSON file

# edit: it was noticed that the places list somehow contained the string elements 'A','D','Via'. Therefore removing it. 

for i in ['A','D','Via']:
    if i in places:
        places.remove(i)

with open("places.json", "w") as outfile: 
            json.dump(sorted(places), outfile)